In [ ]:
#| default_exp models.radga

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
#| export
import torch, re, inspect, pickle, os, torch.nn as nn, math
import torch.nn.functional as F
from dataclasses import dataclass
from typing import Optional, List, Tuple, Mapping, Any, Union
from transformers import (
    PretrainedConfig,
    DistilBertForMaskedLM,
    DistilBertModel,
    DistilBertPreTrainedModel,
)
from transformers.utils.generic import ModelOutput
from transformers.activations import get_activation

from fastcore.meta import *
from fastcore.utils import *

from xcai.losses import *
from xcai.core import store_attr
from xcai.learner import XCDataParallel

comet_ml is installed but `COMET_API_KEY` is not set.


In [ ]:
from nbdev.showdoc import *
import nbdev; nbdev.nbdev_export()

In [ ]:
from transformers import AutoConfig
from xcai.block import *

## Setup

In [ ]:
os.environ['CUDA_VISIBLE_DEVICES'] = "0,1"

In [ ]:
data_dir = '/home/scai/phd/aiz218323/Projects/XC/data/'

In [ ]:
block = XCBlock.from_cfg(data_dir, 'data_metas', tfm='rm', tokenizer='distilbert-base-uncased', 
                         smp_features=[('lbl2data|cat2lbl2data|hlk2lbl2data',1, (1,1,3)), ('cat2data',1, 1), ('hlk2data',1,3)])

/home/scai/phd/aiz218323/.local/lib/python3.9/site-packages/xclib-0.97-py3.9-linux-x86_64.egg/xclib/data/data_utils.py:263: UserWarning: Header mis-match from inferred shape!
  warnings.warn("Header mis-match from inferred shape!")


In [ ]:
data_dir = '/home/scai/phd/aiz218323/scratch/datasets/'
pkl_file = f'{data_dir}/processed/wikiseealso_data-metas_distilbert-base-uncased_rm_radga-final.pkl'

In [ ]:
with open(pkl_file, 'wb') as file: pickle.dump(block, file)

In [ ]:
with open(pkl_file, 'rb') as file: block = pickle.load(file)

In [ ]:
batch = block.train.one_batch(5)
for i,batch in enumerate(block.train.dl):
    if i > 2: break

## Helper

In [ ]:
#| export
@dataclass
class RADOutput(ModelOutput):
    loss: Optional[torch.FloatTensor] = None
    logits: Optional[torch.FloatTensor] = None
    data_repr: Optional[torch.FloatTensor] = None
    data_fused_repr: Optional[torch.FloatTensor] = None
    lbl2data_repr: Optional[torch.FloatTensor] = None
    lbl2data_fused_repr: Optional[torch.FloatTensor] = None
        

In [ ]:
#| export
@dataclass
class EncoderOutput(ModelOutput):
    rep: Optional[torch.FloatTensor] = None
    fused_rep: Optional[torch.FloatTensor] = None
    logits: Optional[torch.FloatTensor] = None
    fusion_weights: Optional[torch.FloatTensor] = None
    meta_repr: Optional[torch.FloatTensor] = None
        

In [ ]:
#| export
class Pooling:

    @staticmethod
    def mean_pooling(data_embeds:torch.FloatTensor, data_attention_mask:torch.LongTensor):
        data_attention_mask = data_attention_mask.unsqueeze(2).expand(data_embeds.size()).float()
        return torch.sum(data_embeds * data_attention_mask, 1) / torch.clamp(data_attention_mask.sum(1), min=1e-9)


## CrossAttention

In [ ]:
#| export
class CrossAttention(nn.Module):
    
    def __init__(self, config: PretrainedConfig):
        super().__init__()
        self.config, self.n_h, self.dim = config, config.n_heads, config.dim
        self.dropout = nn.Dropout(p=config.attention_dropout)

        if self.dim % self.n_h != 0:
            raise ValueError(f"self.n_heads: {self.n_h} must divide self.dim: {self.dim} evenly.")
            
        self.q = nn.Linear(in_features=config.dim, out_features=config.dim)
        self.k = nn.Linear(in_features=config.dim, out_features=config.dim)
        self.v = nn.Linear(in_features=config.dim, out_features=config.dim)
        self.o = nn.Linear(in_features=config.dim, out_features=config.dim)

    def post_init(self):
        self.q.weight.data = torch.eye(self.q.out_features, self.q.in_features, dtype=self.q.weight.dtype)
        self.k.weight.data = torch.eye(self.k.out_features, self.k.in_features, dtype=self.k.weight.dtype)
        self.v.weight.data = torch.eye(self.v.out_features, self.v.in_features, dtype=self.v.weight.dtype)
        self.o.weight.data = torch.eye(self.o.out_features, self.o.in_features, dtype=self.o.weight.dtype)

    def forward(
        self, 
        q: torch.Tensor,
        q_m: torch.Tensor,
        k: torch.Tensor, 
        k_m: torch.Tensor,
        output_attentions:Optional[bool] = False,
    ):
        bs, q_len, dim = q.size()
        v, k_len = k, k.size(1) 

        h_dim = self.dim//self.n_h

        def shape(x: torch.Tensor): return x.view(bs, -1, self.n_h, h_dim).transpose(1, 2)

        def unshape(x: torch.Tensor): return x.transpose(1, 2).contiguous().view(bs, -1, self.n_h * h_dim)

        q = shape(self.q(q))  # (bs, n_h, q_len, h_dim)
        k = shape(self.k(k))  # (bs, n_h, k_len, h_dim)
        v = shape(self.v(v))  # (bs, n_h, k_len, h_dim)

        q = q / math.sqrt(h_dim)  # (bs, n_h, q_len, h_dim)
        sc = torch.matmul(q, k.transpose(2, 3))  # (bs, n_h, q_len, k_len)
        
        q_m, k_m = q_m.view(bs, 1, -1, 1).to(q.dtype), k_m.view(bs, 1, 1, -1).to(q.dtype)
        mask = torch.matmul(q_m, k_m).expand_as(sc)  # (bs, n_h, q_len, k_len)
        
        sc = sc.masked_fill(mask.bool(), torch.tensor(torch.finfo(sc.dtype).min))  # (bs, n_h, q_len, k_len)

        w = nn.functional.softmax(sc, dim=-1)  # (bs, n_h, q_len, k_len)
        w = self.dropout(w)  # (bs, n_h, q_len, k_len)

        o = self.o(unshape(torch.matmul(w, v))) # (bs, q_len, dim)
        
        if output_attentions: return (o, w)
        else: return (o,)
        

### Example

In [ ]:
config = AutoConfig.from_pretrained('distilbert-base-uncased')
fuser = CrossAttention(config)

/opt/conda/envs/ptca/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [ ]:
bsz, data_seq_len, meta_seq_len, dim, dtype = 10, 14, 17, config.dim, torch.float32
data, meta = torch.randn(bsz, data_seq_len, dim, dtype=dtype), torch.randn(bsz, meta_seq_len, dim, dtype=dtype)
data_mask = torch.randint(0, 2, size=(bsz,data_seq_len), dtype=dtype)
meta_mask = torch.randint(0, 2, size=(bsz,meta_seq_len), dtype=dtype)

In [ ]:
o = fuser(data, data_mask, meta, meta_mask)

In [ ]:
o[0].shape

torch.Size([10, 14, 768])

## Blocks

In [ ]:
#| export
class RepresentationHead(nn.Module):
    
    def __init__(self, config):
        super().__init__()
        self.transform = nn.Linear(config.dim, config.dim)
        self.layer_norm = nn.LayerNorm(config.dim, eps=1e-12)
        self.projector = nn.Linear(config.dim, config.dim)
        
        self.post_init()
        
    def post_init(self):
        self.transform.weight.data = torch.eye(self.transform.out_features, self.transform.in_features, 
                                               dtype=self.transform.weight.dtype)
        self.projector.weight.data = torch.eye(self.projector.out_features, self.projector.in_features, 
                                               dtype=self.projector.weight.dtype)
        
    def forward(self, x:torch.Tensor):
        x = self.transform(x)
        x = self.layer_norm(x)
        x = self.projector(x)
        return x
    

In [ ]:
#| export
class GenerationHead(nn.Module):
    
    def __init__(self, config):
        super().__init__()
        self.transform = nn.Linear(config.dim, config.dim)
        self.layer_norm = nn.LayerNorm(config.dim, eps=1e-12)
        self.projector = nn.Linear(config.dim, config.vocab_size)
        self.activation = get_activation(config.activation)
        
    def forward(self, x:torch.Tensor):
        x = self.transform(x)
        x = self.activation(x)
        x = self.layer_norm(x)
        x = self.projector(x)
        return x
    

### Example

In [ ]:
config = AutoConfig.from_pretrained('distilbert-base-uncased')
x = torch.randn(10, 20, config.dim)

In [ ]:
m = RepresentationHead(config)

In [ ]:
m = GenerationHead(config)

## Parameters

In [ ]:
#| export
class Parameters:
    
    @staticmethod
    def from_meta_aug_prefix(prefix:str, **kwargs):
        inputs = {}
        args = [arg for arg in kwargs if prefix is not None and re.match(f'^{prefix}.*_(input_ids|attention_mask|data2ptr|meta_repr)$', arg)]
        for arg in args:
            meta,param = arg.split('_', maxsplit=1)
            inputs.setdefault(meta, {})[param] = kwargs[arg]
        return inputs
    
    @staticmethod
    def from_feat_meta_aug_prefix(feat:str, prefix:str, **kwargs):
        keys = ['attention_mask', 'input_ids', 'meta_repr']
        
        inputs = {f'{prefix}_{k}': kwargs[f'{prefix}_{k}'] for k in keys if f'{prefix}_{k}' in kwargs}
        if prefix is not None and f'{prefix}_{feat}2ptr' in kwargs:
            inputs.update({f'{prefix}_data2ptr': kwargs[f'{prefix}_{feat}2ptr']})
        return inputs
    
    @staticmethod
    def from_meta_pred_prefix(prefix:str, **kwargs):
        inputs = {}
        args = [arg for arg in kwargs if prefix is not None and re.match(f'^[p]?{prefix}.*', arg)]
        for arg in args:
            meta,param = arg.split('_', maxsplit=1)
            if arg[0] == 'p': 
                inputs.setdefault(meta[1:], {})[f'p{param}'] = kwargs[arg]
            else: 
                inputs.setdefault(meta, {})[param] = kwargs[arg]
        return inputs

    @staticmethod
    def get_meta_loss_weights(lw:Union[float,List], n_meta:int):
        if isinstance(lw, float):
            lw = lw/n_meta if n_meta else None
            return [lw] * n_meta
        else:
            if len(lw) != n_meta: raise ValueError(f'length of `lw` should be equal to number of metadata.')
            return lw
        

## Encoder

In [ ]:
#| export
class Encoder(DistilBertPreTrainedModel):
    
    def __init__(self, config:PretrainedConfig, use_noise:Optional[bool]=True, resize_length:Optional[int]=None):
        super().__init__(config)
        store_attr('use_noise')
        
        self.dr_head = RepresentationHead(config)
        self.meta_head = RepresentationHead(config)
        self.gen_head = GenerationHead(config)
        self.cross_head = CrossAttention(config)
        
        self.layer_norm = nn.LayerNorm(config.dim, eps=1e-12)
         
        self.ones = torch.ones(resize_length, dtype=torch.long, device=self.device) if resize_length is not None else None
        self.post_init()
        
    def get_position_embeddings(self) -> nn.Embedding:
        return self.distilbert.get_position_embeddings()
    
    def resize_position_embeddings(self, new_num_position_embeddings: int):
        self.distilbert.resize_position_embeddings(new_num_position_embeddings)
        
    def get_output_embeddings(self) -> nn.Module:
        return self.gen_head.projector

    def set_output_embeddings(self, new_embeddings: nn.Module):
        self.gen_head.projector = new_embeddings
        
    def resize(self, inputs:torch.Tensor, mask:torch.Tensor, num_inputs:torch.Tensor):
        if torch.any(num_inputs == 0): raise ValueError("`num_inputs` should be non-zero positive integer.")
        bsz, dim, total_num_inputs = num_inputs.shape[0], inputs.shape[-1], inputs.shape[0]
        
        self.ones = self.ones.to(inputs.device)
        ones = (
            torch.ones(total_num_inputs, dtype=torch.long, device=inputs.device) 
            if self.ones is None or self.ones.shape[0] < total_num_inputs else self.ones[:total_num_inputs]
        )

        max_num_inputs = num_inputs.max()
        xnum_inputs = max_num_inputs-num_inputs+1

        inputs_ptr = num_inputs.cumsum(dim=0)-1
        repeat_inputs = ones.scatter(0, inputs_ptr, xnum_inputs)
        
        resized_inputs = inputs.repeat_interleave(repeat_inputs, dim=0)
        resized_mask = mask.repeat_interleave(repeat_inputs, dim=0)
        
        ignore_mask_idx = ones.scatter(0, inputs_ptr, 0).repeat_interleave(repeat_inputs, dim=0).view(bsz, -1)
        ignore_mask_idx[:, -1] = 1; ignore_mask_idx = ignore_mask_idx.view(-1, 1)
        
        resized_mask *= ignore_mask_idx

        return resized_inputs,resized_mask
    
    def get_noise(self, inputs:torch.Tensor, mask:torch.Tensor, num_inputs:torch.Tensor):
        if torch.any(num_inputs == 0): raise ValueError("`num_inputs` should be non-zero positive integer.")
        rnd_idx = torch.randperm(len(num_inputs))
        last_idx = num_inputs.cumsum(dim=0) - 1
        return inputs[last_idx][rnd_idx], mask[last_idx][rnd_idx]
    
    def add_noise(self, inputs:torch.Tensor, mask:torch.Tensor, noise_inputs:torch.Tensor, noise_mask:torch.Tensor):
        bsz, seq_len = noise_inputs.shape[0], inputs.shape[-1]
        
        inputs = torch.concat([inputs.view(bsz,-1,seq_len), noise_inputs.view(bsz,1,-1)], dim=1)
        mask = torch.concat([mask.view(bsz,-1,seq_len), noise_mask.view(bsz,1,-1)], dim=1)
        
        return inputs.view(-1, seq_len), mask.view(-1, seq_len)
        
        
    
        
        
        
    def encode(self, input_ids:torch.Tensor, attention_mask:torch.Tensor, **kwargs):
        return self.distilbert(
            input_ids=input_ids,
            attention_mask=attention_mask,
            **kwargs
        )
    
    def dr(self, embed:torch.Tensor, attention_mask:torch.Tensor):
        embed = self.dr_head(embed)
        return F.normalize(Pooling.mean_pooling(embed, attention_mask), dim=1)

    def meta(self, embed:torch.Tensor, attention_mask:torch.Tensor):
        embed = self.meta_head(embed)
        return F.normalize(Pooling.mean_pooling(embed, attention_mask), dim=1)
    
    def meta_unnormalized(self, embed:torch.Tensor, attention_mask:torch.Tensor):
        embed = self.meta_head(embed)
        return Pooling.mean_pooling(embed, attention_mask)
    
    def gen(self, x:torch.Tensor):
        return self.gen_head(x)
    
    


    
    def fuse_meta_into_embeddings(self, embed:torch.Tensor, attention_mask:torch.Tensor, prefix:str, **kwargs):
        meta_kwargs = Parameters.from_meta_aug_prefix(prefix, **kwargs)
        meta_repr, weights, performed_fusion = {}, [], False
        
        for m_key, m_args in meta_kwargs.items():
            idx = torch.where(m_args['data2ptr'] > 0)[0]
            meta_repr[m_key] = torch.empty(0, self.config.dim).to(embed)
            if len(idx):
                performed_fusion = True
                if 'meta_repr' in m_args:
                    m_repr,m_repr_mask = m_args['meta_repr'],torch.any(m_args['attention_mask'], dim=1).long().view(-1,1)
                    m_repr,m_repr_mask = self.resize(m_repr, m_repr_mask, m_args['data2ptr'][idx])
                    m_repr_mask = m_repr_mask.bool()
                else:
                    m_input_ids, m_attention_mask = self.resize(m_args['input_ids'], m_args['attention_mask'], 
                                                                m_args['data2ptr'][idx])

                    if self.use_noise:
                        n_input_ids, n_attention_mask = self.get_noise(m_args['input_ids'], m_args['attention_mask'], 
                                                                       m_args['data2ptr'][idx])
                        m_input_ids, m_attention_mask = self.add_noise(m_input_ids, m_attention_mask, 
                                                                       n_input_ids, n_attention_mask)

                    m_embed = self.encode(m_input_ids, m_attention_mask)[0]

                    m_repr = self.meta_unnormalized(m_embed, m_attention_mask)
                    m_repr_mask = torch.any(m_attention_mask, dim=1)
                    
                m_repr, m_repr_mask = m_repr.view(len(idx), -1, self.config.dim), m_repr_mask.view(len(idx), -1)
                
                meta_repr[m_key] = m_repr[:, :-1][m_repr_mask[:, :-1]] if self.use_noise else m_repr[m_repr_mask]
                meta_repr[m_key] = F.normalize(meta_repr[m_key], dim=1)
                
                fused_embed, w = self.cross_head(embed[idx], attention_mask[idx], m_repr, m_repr_mask, output_attentions=True)
                embed[idx] += fused_embed
                weights.append(w)
        
        if performed_fusion: embed = self.layer_norm(embed)        
        return embed, weights, meta_repr
    
    def forward(
        self, 
        data_input_ids: torch.Tensor, 
        data_attention_mask: torch.Tensor,
        data_aug_meta_prefix: Optional[str]=None,
        data_gen_idx:Optional[torch.Tensor]=None,
        data_type:Optional[str]=None,
        data_unnormalized:Optional[bool]=False,
        **kwargs
    ):
        data_o = self.encode(data_input_ids, data_attention_mask)
        
        if data_type is not None and data_type == "meta":
            data_repr = self.meta_unnormalized(data_o[0], data_attention_mask) if data_unnormalized else self.meta(data_o[0], data_attention_mask)
        else: 
            data_repr = self.dr(data_o[0], data_attention_mask)
        
        data_fused_repr = data_fused_logits = fusion_weights = meta_repr = None
        if data_aug_meta_prefix is not None:
            data_fused_embed, fusion_weights, meta_repr = self.fuse_meta_into_embeddings(data_o[0], 
                                                                                         data_attention_mask, 
                                                                                         data_aug_meta_prefix, 
                                                                                         **kwargs)
            data_fused_repr = self.dr(data_fused_embed, data_attention_mask)
            data_fused_logits = self.gen(data_fused_embed if data_gen_idx is None else data_fused_embed[data_gen_idx])
        
        return EncoderOutput(
            rep=data_repr,
            fused_rep=data_fused_repr,
            logits=data_fused_logits,
            
            meta_repr=meta_repr,
        )
        

## `RAD001`

In [ ]:
#| export
class RAD001(DistilBertForMaskedLM):
    use_generation,use_representation = True,True
    _tied_weights_keys = ["encoder.distilbert", "encoder.gen_head.transform", "encoder.gen_head.layer_norm", 
                          "encoder.gen_head.projector"]
    
    def __init__(
        self, config,
        
        num_batch_labels:Optional[int]=None, 
        ignore_token:Optional[int]=0,
        batch_size:Optional[int]=None,
        margin:Optional[int]=0.3,
        num_negatives:Optional[int]=5,
        tau:Optional[float]=0.1,
        apply_softmax:Optional[bool]=True,
        
        data_aug_meta_prefix:Optional[List]=None, 
        lbl2data_aug_meta_prefix:Optional[List]=None, 
        resize_length:Optional[int]=None,
        
        gen_loss_weight:Optional[float]=0.01,
        meta_loss_weight:Optional[Union[List,float]]=0.3, 
        pred_meta_prefix:Optional[List]=None,
        
        fusion_loss_weight:Optional[int]=0.1,
        use_fusion_loss:Optional[bool]=True,
        use_noise:Optional[bool]=True,
        
        use_encoder_parallel:Optional[bool]=True,
    ):
        super().__init__(config)
        self.lw, self.m_lw, self.f_lw = gen_loss_weight, meta_loss_weight, fusion_loss_weight
        store_attr('pred_meta_prefix,data_aug_meta_prefix,lbl2data_aug_meta_prefix,use_fusion_loss,use_encoder_parallel')
        
        self.encoder = Encoder(config, use_noise=use_noise, resize_length=resize_length)
        
        self.post_init()
        
        self.gen_loss_fn = MultiCrossEntropy(tn_targ=num_batch_labels, ig_tok=ignore_token, reduce='mean')
        self.rep_loss_fn = MultiTriplet(bsz=batch_size, tn_targ=num_batch_labels, margin=margin, 
                                        n_negatives=num_negatives, tau=tau, apply_softmax=apply_softmax, 
                                        reduce='mean')
        
        self.remap_post_init()
        self.init_retrieval_head()
        
    def init_retrieval_head(self):
        self.encoder.dr_head.post_init()
        self.encoder.meta_head.post_init()
            
    def init_generation_head(self):
        self.encoder.gen_head.projector.weight.data = self.get_input_embeddings().weight.data.clone()
        
    def remap_post_init(self):
        self.encoder.distilbert = self.distilbert 
        self.encoder.gen_head.transform = self.vocab_transform
        self.encoder.gen_head.layer_norm = self.vocab_layer_norm
        self.encoder.gen_head.projector = self.vocab_projector
        
        
        
        
    def compute_loss(self, inp_logits, inp_repr, targ_logits, targ_repr, 
                     inp_input_ids, targ_input_ids, targ_ptr, targ_idx, ptarg_ptr, ptarg_idx):
        
        gen_loss = self.gen_loss_fn(inp_logits, targ_input_ids, targ_ptr) + self.gen_loss_fn(targ_logits, inp_input_ids)
        
        rep_loss = self.rep_loss_fn(inp_repr, targ_repr, targ_ptr, targ_idx, ptarg_ptr, ptarg_idx)
        
        return rep_loss + self.lw * gen_loss
    
    def compute_meta_loss(self, data_repr, lbl2data_repr, **kwargs):
        if self.use_encoder_parallel: 
            encoder = XCDataParallel(module=self.encoder)
        else: encoder = self.encoder
            
        meta_inputs = Parameters.from_meta_pred_prefix(self.pred_meta_prefix, **kwargs)
        m_lw = Parameters.get_meta_loss_weights(self.m_lw, len(meta_inputs))
        
        loss = 0.0
        for inputs,lw in zip(meta_inputs.values(), m_lw):
            if 'lbl2data2ptr' in inputs:
                idx = torch.where(inputs['lbl2data2ptr'])[0]
                if len(idx) > 0:
                    inputs_o = encoder(data_input_ids=inputs['input_ids'],
                                       data_attention_mask=inputs['attention_mask'], data_type="meta")
                    m_loss = self.rep_loss_fn(lbl2data_repr[idx], inputs_o.rep, inputs['data2ptr'][idx],
                                              inputs['idx'], inputs['pdata2ptr'][idx], inputs['pidx'])
                    loss += lw * m_loss

            elif 'data2ptr' in inputs:
                idx = torch.where(inputs['data2ptr'])[0]
                if len(idx) > 0:
                    inputs_o = encoder(data_input_ids=inputs['input_ids'], 
                                       data_attention_mask=inputs['attention_mask'], data_type="meta")
                    m_loss = self.rep_loss_fn(data_repr[idx], inputs_o.rep, inputs['data2ptr'][idx], inputs['idx'], 
                                              inputs['pdata2ptr'][idx], inputs['pidx'])
                    loss += lw * m_loss       

            else: raise ValueError('Invalid metadata input arguments.')
        return loss
    
    def compute_fusion_loss(self, data_repr, meta_repr:Dict, prefix:str, **kwargs):
        meta_inputs = Parameters.from_meta_pred_prefix(prefix, **kwargs)
        
        loss = 0.0
        for key,input_repr in meta_repr.items():
            inputs = meta_inputs[key]
            if 'lbl2data2ptr' in inputs:
                idx = torch.where(inputs['lbl2data2ptr'])[0]
                if len(idx) > 0:
                    m_loss = self.rep_loss_fn(data_repr[idx], input_repr, inputs['data2ptr'][idx],
                                              inputs['idx'], inputs['pdata2ptr'][idx], inputs['pidx'])
                    loss += self.f_lw * m_loss

            elif 'data2ptr' in inputs:
                idx = torch.where(inputs['data2ptr'])[0]
                if len(idx) > 0:
                    m_loss = self.rep_loss_fn(data_repr[idx], input_repr, inputs['data2ptr'][idx], inputs['idx'], 
                                              inputs['pdata2ptr'][idx], inputs['pidx'])
                    loss += self.f_lw * m_loss       

            else: raise ValueError('Invalid metadata input arguments.')
        return loss
    
        

    def disable_noise(self):
        use_noise = self.encoder.module.use_noise if isinstance(self.encoder, XCDataParallel) else self.encoder.use_noise
        if isinstance(self.encoder, XCDataParallel): self.encoder.module.use_noise = False
        else: self.encoder.use_noise = False
        return use_noise
    
    def set_noise(self, use_noise):
        if isinstance(self.encoder, XCDataParallel): self.encoder.module.use_noise = use_noise
        else: self.encoder.use_noise = use_noise
            
    def get_noise(self):
        return self.encoder.module.use_noise if isinstance(self.encoder, XCDataParallel) else self.encoder.use_noise

    def get_meta_representation(
        self,
        data_input_ids:Optional[torch.Tensor]=None,
        data_attention_mask:Optional[torch.Tensor]=None,
        **kwargs
    ):
        if self.use_encoder_parallel: 
            encoder = XCDataParallel(module=self.encoder)
        else: encoder = self.encoder
            
        data_o = encoder(data_input_ids=data_input_ids, data_attention_mask=data_attention_mask, 
                         data_unnormalized=True, data_type="meta")
        return RADOutput(
            logits=data_o.logits,
            data_repr=data_o.rep,
            data_fused_repr=data_o.fused_rep,
        )
        
        
    def get_last_item_mask(self, num_input:torch.Tensor, input_sz:int):
        idx = torch.where(num_input > 0)[0]
        input_ptr = num_input[idx].cumsum(dim=0)-1
        return torch.zeros(input_sz, dtype=torch.bool, device=num_input.device).scatter(0, input_ptr, 1)


    def forward(
        self,
        data_input_ids:Optional[torch.Tensor]=None,
        data_attention_mask:Optional[torch.Tensor]=None,
        lbl2data_data2ptr:Optional[torch.Tensor]=None,
        lbl2data_idx:Optional[torch.Tensor]=None,
        lbl2data_input_ids:Optional[torch.Tensor]=None,
        lbl2data_attention_mask:Optional[torch.Tensor]=None,
        plbl2data_data2ptr:Optional[torch.Tensor]=None,
        plbl2data_idx:Optional[torch.Tensor]=None,
        output_attentions: Optional[bool] = None,
        output_hidden_states: Optional[bool] = None,
        return_dict: Optional[bool] = None,
        **kwargs
    ):  
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict
        
        if self.use_encoder_parallel: 
            encoder = XCDataParallel(module=self.encoder)
        else: encoder = self.encoder
        
        data_meta_kwargs = Parameters.from_feat_meta_aug_prefix('data', self.data_aug_meta_prefix, **kwargs)
        data_o = encoder(data_input_ids=data_input_ids, data_attention_mask=data_attention_mask, 
                         data_aug_meta_prefix=self.data_aug_meta_prefix, **data_meta_kwargs)
        
        
        loss = None; lbl2data_o = EncoderOutput()
        if lbl2data_input_ids is not None:
            lbl2data_gen_idx = self.get_last_item_mask(lbl2data_data2ptr, len(lbl2data_idx))
            lbl2data_meta_kwargs = Parameters.from_feat_meta_aug_prefix('lbl2data', self.lbl2data_aug_meta_prefix, **kwargs)
            
            lbl2data_o = encoder(data_input_ids=lbl2data_input_ids, data_attention_mask=lbl2data_attention_mask, 
                                 data_aug_meta_prefix=self.lbl2data_aug_meta_prefix, data_gen_idx=lbl2data_gen_idx,
                                 **lbl2data_meta_kwargs)
            
            loss = self.compute_loss(data_o.logits, data_o.fused_rep, lbl2data_o.logits, lbl2data_o.fused_rep, 
                                     data_input_ids,lbl2data_input_ids,lbl2data_data2ptr,lbl2data_idx,
                                     plbl2data_data2ptr,plbl2data_idx)
            
            loss += self.compute_meta_loss(data_o.fused_rep, lbl2data_o.fused_rep, **kwargs)
            
            if self.use_fusion_loss:
                loss += self.compute_fusion_loss(data_o.fused_rep, data_o.meta_repr, self.data_aug_meta_prefix, **kwargs)
                loss += self.compute_fusion_loss(lbl2data_o.fused_rep, lbl2data_o.meta_repr, self.lbl2data_aug_meta_prefix, **kwargs)
            
        if not return_dict:
            o = (data_o.logits,data_o.rep,data_o.fused_rep,lbl2data_o.logits,lbl2data_o.rep,lbl2data_o.fused_rep)
            return ((loss,) + o) if loss is not None else o
        
        
        return RADOutput(
            loss=loss,
            
            logits=data_o.logits,
            data_repr=data_o.rep,
            data_fused_repr=data_o.fused_rep,
            
            lbl2data_repr=lbl2data_o.rep,
            lbl2data_fused_repr=lbl2data_o.fused_rep,
        )
        

### Example

In [ ]:
model = RAD001.from_pretrained('distilbert-base-uncased', num_batch_labels=5000, ignore_token=0, batch_size=100,
                               margin=0.3, num_negatives=5, tau=0.1, apply_softmax=True,
                               
                               data_aug_meta_prefix='hlk2data', lbl2data_aug_meta_prefix='hlk2lbl', 
                               resize_length=5000,
                               
                               gen_loss_weight=0.001, meta_loss_weight=0.3, pred_meta_prefix='cat', 
                               
                               fusion_loss_weight=0.05, tie_word_embeddings=False,
                               
                               use_fusion_loss=True, use_noise=True, use_encoder_parallel=False)

Some weights of RAD001 were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['encoder.cross_head.k.bias', 'encoder.cross_head.k.weight', 'encoder.cross_head.o.bias', 'encoder.cross_head.o.weight', 'encoder.cross_head.q.bias', 'encoder.cross_head.q.weight', 'encoder.cross_head.v.bias', 'encoder.cross_head.v.weight', 'encoder.dr_head.layer_norm.bias', 'encoder.dr_head.layer_norm.weight', 'encoder.dr_head.projector.bias', 'encoder.dr_head.projector.weight', 'encoder.dr_head.transform.bias', 'encoder.dr_head.transform.weight', 'encoder.gen_head.projector.weight', 'encoder.linear_layer.bias', 'encoder.linear_layer.weight', 'encoder.meta_head.layer_norm.bias', 'encoder.meta_head.layer_norm.weight', 'encoder.meta_head.projector.bias', 'encoder.meta_head.projector.weight', 'encoder.meta_head.transform.bias', 'encoder.meta_head.transform.weight', 'vocab_projector.weight']
You should probably TRAIN this model on a down-stream task to be able to use

In [ ]:
model.init_retrieval_head()
model.init_generation_head()

In [ ]:
model = model.to('cuda')

In [ ]:
b = prepare_batch(model, batch, m_args=[
    'pcat2data_idx', 'pcat2data_data2ptr', 'cat2data_idx', 'cat2data_input_ids', 'cat2data_attention_mask', 
    'cat2data_data2ptr',
    'pcat2lbl_idx', 'pcat2lbl_lbl2data2ptr', 'pcat2lbl_data2ptr', 'cat2lbl_idx', 'cat2lbl_input_ids', 
    'cat2lbl_attention_mask', 'cat2lbl_lbl2data2ptr', 'cat2lbl_data2ptr',
    
    'phlk2data_idx', 'phlk2data_data2ptr', 'hlk2data_idx', 'hlk2data_input_ids', 'hlk2data_attention_mask', 
    'hlk2data_data2ptr',
    'phlk2lbl_idx', 'phlk2lbl_lbl2data2ptr', 'phlk2lbl_data2ptr', 'hlk2lbl_idx', 'hlk2lbl_input_ids', 
    'hlk2lbl_attention_mask', 'hlk2lbl_lbl2data2ptr', 'hlk2lbl_data2ptr',
])

In [ ]:
o = model(**b.to(model.device))

In [ ]:
o.loss

tensor(0.0928, device='cuda:0', grad_fn=<AddBackward0>)

## `RAD002`

In [ ]:
#| export
class Encoder002(DistilBertPreTrainedModel):
    
    def __init__(self, config:PretrainedConfig, use_noise:Optional[bool]=True, resize_length:Optional[int]=None):
        super().__init__(config)
        store_attr('use_noise')
        self.distilbert = DistilBertModel(config)
        
        self.dr_head = RepresentationHead(config)
        self.meta_head = RepresentationHead(config)
        self.cross_head = CrossAttention(config)
         
        self.ones = torch.ones(resize_length, dtype=torch.long, device=self.device) if resize_length is not None else None
        self.post_init()
        
    def get_position_embeddings(self) -> nn.Embedding:
        return self.distilbert.get_position_embeddings()
    
    def resize_position_embeddings(self, new_num_position_embeddings: int):
        self.distilbert.resize_position_embeddings(new_num_position_embeddings)
        
    def resize(self, inputs:torch.Tensor, mask:torch.Tensor, num_inputs:torch.Tensor):
        if torch.any(num_inputs == 0): raise ValueError("`num_inputs` should be non-zero positive integer.")
        bsz, dim, total_num_inputs = num_inputs.shape[0], inputs.shape[-1], inputs.shape[0]
        
        self.ones = self.ones.to(inputs.device)
        ones = (
            torch.ones(total_num_inputs, dtype=torch.long, device=inputs.device) 
            if self.ones is None or self.ones.shape[0] < total_num_inputs else self.ones[:total_num_inputs]
        )

        max_num_inputs = num_inputs.max()
        xnum_inputs = max_num_inputs-num_inputs+1

        inputs_ptr = num_inputs.cumsum(dim=0)-1
        repeat_inputs = ones.scatter(0, inputs_ptr, xnum_inputs)
        
        resized_inputs = inputs.repeat_interleave(repeat_inputs, dim=0)
        resized_mask = mask.repeat_interleave(repeat_inputs, dim=0)
        
        ignore_mask_idx = ones.scatter(0, inputs_ptr, 0).repeat_interleave(repeat_inputs, dim=0).view(bsz, -1)
        ignore_mask_idx[:, -1] = 1; ignore_mask_idx = ignore_mask_idx.view(-1, 1)
        
        resized_mask *= ignore_mask_idx

        return resized_inputs,resized_mask
    
    def get_noise(self, inputs:torch.Tensor, mask:torch.Tensor, num_inputs:torch.Tensor):
        if torch.any(num_inputs == 0): raise ValueError("`num_inputs` should be non-zero positive integer.")
        rnd_idx = torch.randperm(len(num_inputs))
        last_idx = num_inputs.cumsum(dim=0) - 1
        return inputs[last_idx][rnd_idx], mask[last_idx][rnd_idx]
    
    def add_noise(self, inputs:torch.Tensor, mask:torch.Tensor, noise_inputs:torch.Tensor, noise_mask:torch.Tensor):
        bsz, seq_len = noise_inputs.shape[0], inputs.shape[-1]
        
        inputs = torch.concat([inputs.view(bsz,-1,seq_len), noise_inputs.view(bsz,1,-1)], dim=1)
        mask = torch.concat([mask.view(bsz,-1,seq_len), noise_mask.view(bsz,1,-1)], dim=1)
        
        return inputs.view(-1, seq_len), mask.view(-1, seq_len)
        
        
    
        
    def encode(self, input_ids:torch.Tensor, attention_mask:torch.Tensor, **kwargs):
        return self.distilbert(
            input_ids=input_ids,
            attention_mask=attention_mask,
            **kwargs
        )
    
    def dr(self, embed:torch.Tensor, attention_mask:torch.Tensor):
        embed = self.dr_head(embed)
        return F.normalize(Pooling.mean_pooling(embed, attention_mask), dim=1)

    def meta(self, embed:torch.Tensor, attention_mask:torch.Tensor):
        embed = self.meta_head(embed)
        return F.normalize(Pooling.mean_pooling(embed, attention_mask), dim=1)
    
    def meta_unnormalized(self, embed:torch.Tensor, attention_mask:torch.Tensor):
        embed = self.meta_head(embed)
        return Pooling.mean_pooling(embed, attention_mask)
    


    
    def fuse_meta_into_embeddings(self, embed:torch.Tensor, attention_mask:torch.Tensor, prefix:str, **kwargs):
        meta_kwargs = Parameters.from_meta_aug_prefix(prefix, **kwargs)
        meta_repr, weights = {}, []
        
        for m_key, m_args in meta_kwargs.items():
            idx = torch.where(m_args['data2ptr'] > 0)[0]
            meta_repr[m_key] = torch.empty(0, self.config.dim).to(embed)
            
            if len(idx):
                if 'meta_repr' in m_args:
                    m_repr,m_repr_mask = m_args['meta_repr'],torch.any(m_args['attention_mask'], dim=1).long().view(-1,1)
                    m_repr,m_repr_mask = self.resize(m_repr, m_repr_mask, m_args['data2ptr'][idx])
                    m_repr_mask = m_repr_mask.bool()
                else:
                    m_input_ids, m_attention_mask = self.resize(m_args['input_ids'], m_args['attention_mask'], 
                                                                m_args['data2ptr'][idx])

                    if self.use_noise:
                        n_input_ids, n_attention_mask = self.get_noise(m_args['input_ids'], m_args['attention_mask'], 
                                                                       m_args['data2ptr'][idx])
                        m_input_ids, m_attention_mask = self.add_noise(m_input_ids, m_attention_mask, 
                                                                       n_input_ids, n_attention_mask)

                    m_embed = self.encode(m_input_ids, m_attention_mask)[0]

                    m_repr = self.meta_unnormalized(m_embed, m_attention_mask)
                    m_repr_mask = torch.any(m_attention_mask, dim=1)
                    
                m_repr, m_repr_mask = m_repr.view(len(idx), -1, self.config.dim), m_repr_mask.view(len(idx), -1)
                
                meta_repr[m_key] = m_repr[:, :-1][m_repr_mask[:, :-1]] if self.use_noise else m_repr[m_repr_mask]
                meta_repr[m_key] = F.normalize(meta_repr[m_key], dim=1)
                
                fused_embed, w = self.cross_head(embed[idx], attention_mask[idx], m_repr, m_repr_mask, output_attentions=True)
                embed[idx] += fused_embed
                weights.append(w)
               
        return embed, weights, meta_repr
        
    
    def forward(
        self, 
        data_input_ids: torch.Tensor, 
        data_attention_mask: torch.Tensor,
        data_aug_meta_prefix: Optional[str]=None,
        data_type:Optional[str]=None,
        data_unnormalized:Optional[bool]=False,
        **kwargs
    ):
        data_o = self.encode(data_input_ids, data_attention_mask)
        
        if data_type is not None and data_type == "meta":
            data_repr = self.meta_unnormalized(data_o[0], data_attention_mask) if data_unnormalized else self.meta(data_o[0], data_attention_mask)
        else: 
            data_repr = self.dr(data_o[0], data_attention_mask)
        
        data_fused_repr = data_fused_logits = fusion_weights = meta_repr = None
        if data_aug_meta_prefix is not None:
            data_fused_embed, fusion_weights, meta_repr = self.fuse_meta_into_embeddings(data_o[0], 
                                                                                         data_attention_mask, 
                                                                                         data_aug_meta_prefix, 
                                                                                         **kwargs)
            data_fused_repr = self.dr(data_fused_embed, data_attention_mask)
        
        return EncoderOutput(
            rep=data_repr,
            fused_rep=data_fused_repr,
            meta_repr=meta_repr,
        )
        

In [ ]:
#| export
class RAD002(DistilBertPreTrainedModel):
    use_generation,use_representation = False,True
    _tied_weights_keys = ["encoder.distilbert"]
    
    def __init__(
        self, config,
        
        num_batch_labels:Optional[int]=None, 
        batch_size:Optional[int]=None,
        margin:Optional[int]=0.3,
        num_negatives:Optional[int]=5,
        tau:Optional[float]=0.1,
        apply_softmax:Optional[bool]=True,
        
        data_aug_meta_prefix:Optional[List]=None, 
        lbl2data_aug_meta_prefix:Optional[List]=None, 
        resize_length:Optional[int]=None,
        
        gen_loss_weight:Optional[float]=0.01,
        meta_loss_weight:Optional[Union[List,float]]=0.3, 
        pred_meta_prefix:Optional[List]=None,
        
        fusion_loss_weight:Optional[int]=0.1,
        use_fusion_loss:Optional[bool]=True,
        use_noise:Optional[bool]=True,
        
        use_encoder_parallel:Optional[bool]=True,
    ):
        super().__init__(config)
        self.lw, self.m_lw, self.f_lw = gen_loss_weight, meta_loss_weight, fusion_loss_weight
        store_attr('pred_meta_prefix,data_aug_meta_prefix,lbl2data_aug_meta_prefix,use_fusion_loss,use_encoder_parallel')
        
        self.encoder = Encoder002(config, use_noise=use_noise, resize_length=resize_length)
        
        self.post_init()
        self.rep_loss_fn = MultiTriplet(bsz=batch_size, tn_targ=num_batch_labels, margin=margin, n_negatives=num_negatives, 
                                        tau=tau, apply_softmax=apply_softmax, reduce='mean')
        self.remap_post_init()
        self.init_retrieval_head()
        self.init_cross_head()
        
    def init_retrieval_head(self):
        self.encoder.dr_head.post_init()
        self.encoder.meta_head.post_init()

    def init_cross_head(self):
        self.encoder.cross_head.post_init()
            
    def remap_post_init(self):
         self.distilbert = self.encoder.distilbert
        
    def compute_loss(self, inp_repr, targ_repr, targ_ptr, targ_idx, ptarg_ptr, ptarg_idx):
        rep_loss = self.rep_loss_fn(inp_repr, targ_repr, targ_ptr, targ_idx, ptarg_ptr, ptarg_idx)
        return rep_loss
    
    def compute_meta_loss(self, data_repr, lbl2data_repr, **kwargs):
        if self.use_encoder_parallel: 
            encoder = XCDataParallel(module=self.encoder)
        else: encoder = self.encoder
            
        meta_inputs = Parameters.from_meta_pred_prefix(self.pred_meta_prefix, **kwargs)
        m_lw = Parameters.get_meta_loss_weights(self.m_lw, len(meta_inputs))
        
        loss = 0.0
        for inputs,lw in zip(meta_inputs.values(), m_lw):
            if 'lbl2data2ptr' in inputs:
                idx = torch.where(inputs['lbl2data2ptr'])[0]
                if len(idx) > 0:
                    inputs_o = encoder(data_input_ids=inputs['input_ids'],
                                       data_attention_mask=inputs['attention_mask'], data_type="meta")
                    m_loss = self.rep_loss_fn(lbl2data_repr[idx], inputs_o.rep, inputs['data2ptr'][idx],
                                              inputs['idx'], inputs['pdata2ptr'][idx], inputs['pidx'])
                    loss += lw * m_loss

            elif 'data2ptr' in inputs:
                idx = torch.where(inputs['data2ptr'])[0]
                if len(idx) > 0:
                    inputs_o = encoder(data_input_ids=inputs['input_ids'], 
                                       data_attention_mask=inputs['attention_mask'], data_type="meta")
                    m_loss = self.rep_loss_fn(data_repr[idx], inputs_o.rep, inputs['data2ptr'][idx], inputs['idx'], 
                                              inputs['pdata2ptr'][idx], inputs['pidx'])
                    loss += lw * m_loss       

            else: raise ValueError('Invalid metadata input arguments.')
        return loss
    
    def compute_fusion_loss(self, data_repr, meta_repr:Dict, prefix:str, **kwargs):
        meta_inputs = Parameters.from_meta_pred_prefix(prefix, **kwargs)
        
        loss = 0.0
        for key,input_repr in meta_repr.items():
            inputs = meta_inputs[key]
            if 'lbl2data2ptr' in inputs:
                idx = torch.where(inputs['lbl2data2ptr'])[0]
                if len(idx) > 0:
                    m_loss = self.rep_loss_fn(data_repr[idx], input_repr, inputs['data2ptr'][idx],
                                              inputs['idx'], inputs['pdata2ptr'][idx], inputs['pidx'])
                    loss += self.f_lw * m_loss

            elif 'data2ptr' in inputs:
                idx = torch.where(inputs['data2ptr'])[0]
                if len(idx) > 0:
                    m_loss = self.rep_loss_fn(data_repr[idx], input_repr, inputs['data2ptr'][idx], inputs['idx'], 
                                              inputs['pdata2ptr'][idx], inputs['pidx'])
                    loss += self.f_lw * m_loss       

            else: raise ValueError('Invalid metadata input arguments.')
        return loss
    
        

    def disable_noise(self):
        use_noise = self.encoder.module.use_noise if isinstance(self.encoder, XCDataParallel) else self.encoder.use_noise
        if isinstance(self.encoder, XCDataParallel): self.encoder.module.use_noise = False
        else: self.encoder.use_noise = False
        return use_noise
    
    def set_noise(self, use_noise):
        if isinstance(self.encoder, XCDataParallel): self.encoder.module.use_noise = use_noise
        else: self.encoder.use_noise = use_noise
            
    def get_noise(self):
        return self.encoder.module.use_noise if isinstance(self.encoder, XCDataParallel) else self.encoder.use_noise

    def get_meta_representation(
        self,
        data_input_ids:Optional[torch.Tensor]=None,
        data_attention_mask:Optional[torch.Tensor]=None,
        **kwargs
    ):
        if self.use_encoder_parallel: 
            encoder = XCDataParallel(module=self.encoder)
        else: encoder = self.encoder
            
        data_o = encoder(data_input_ids=data_input_ids, data_attention_mask=data_attention_mask, 
                         data_unnormalized=True, data_type="meta")
        return RADOutput(
            logits=data_o.logits,
            data_repr=data_o.rep,
            data_fused_repr=data_o.fused_rep,
        )
        
        
    def get_last_item_mask(self, num_input:torch.Tensor, input_sz:int):
        idx = torch.where(num_input > 0)[0]
        input_ptr = num_input[idx].cumsum(dim=0)-1
        return torch.zeros(input_sz, dtype=torch.bool, device=num_input.device).scatter(0, input_ptr, 1)


    def forward(
        self,
        data_input_ids:Optional[torch.Tensor]=None,
        data_attention_mask:Optional[torch.Tensor]=None,
        lbl2data_data2ptr:Optional[torch.Tensor]=None,
        lbl2data_idx:Optional[torch.Tensor]=None,
        lbl2data_input_ids:Optional[torch.Tensor]=None,
        lbl2data_attention_mask:Optional[torch.Tensor]=None,
        plbl2data_data2ptr:Optional[torch.Tensor]=None,
        plbl2data_idx:Optional[torch.Tensor]=None,
        output_attentions: Optional[bool] = None,
        output_hidden_states: Optional[bool] = None,
        return_dict: Optional[bool] = None,
        **kwargs
    ):  
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict
        
        if self.use_encoder_parallel: 
            encoder = XCDataParallel(module=self.encoder)
        else: encoder = self.encoder
        
        data_meta_kwargs = Parameters.from_feat_meta_aug_prefix('data', self.data_aug_meta_prefix, **kwargs)
        data_o = encoder(data_input_ids=data_input_ids, data_attention_mask=data_attention_mask, 
                         data_aug_meta_prefix=self.data_aug_meta_prefix, **data_meta_kwargs)
        
        
        loss = None; lbl2data_o = EncoderOutput()
        if lbl2data_input_ids is not None:
            lbl2data_meta_kwargs = Parameters.from_feat_meta_aug_prefix('lbl2data', self.lbl2data_aug_meta_prefix, **kwargs)
            
            lbl2data_o = encoder(data_input_ids=lbl2data_input_ids, data_attention_mask=lbl2data_attention_mask, 
                                 data_aug_meta_prefix=self.lbl2data_aug_meta_prefix, **lbl2data_meta_kwargs)
            
            loss = self.compute_loss(data_o.fused_rep, lbl2data_o.fused_rep,lbl2data_data2ptr,lbl2data_idx,
                                     plbl2data_data2ptr,plbl2data_idx)
            
            loss += self.compute_meta_loss(data_o.fused_rep, lbl2data_o.fused_rep, **kwargs)
            
            if self.use_fusion_loss:
                loss += self.compute_fusion_loss(data_o.fused_rep, data_o.meta_repr, self.data_aug_meta_prefix, **kwargs)
                loss += self.compute_fusion_loss(lbl2data_o.fused_rep, lbl2data_o.meta_repr, self.lbl2data_aug_meta_prefix, **kwargs)
            
        if not return_dict:
            o = (data_o.logits,data_o.rep,data_o.fused_rep,lbl2data_o.logits,lbl2data_o.rep,lbl2data_o.fused_rep)
            return ((loss,) + o) if loss is not None else o
        
        
        return RADOutput(
            loss=loss,
            
            data_repr=data_o.rep,
            data_fused_repr=data_o.fused_rep,
            
            lbl2data_repr=lbl2data_o.rep,
            lbl2data_fused_repr=lbl2data_o.fused_rep,
        )
        

### Example

In [ ]:
model = RAD002.from_pretrained('distilbert-base-uncased', num_batch_labels=5000, batch_size=100,
                               margin=0.3, num_negatives=5, tau=0.1, apply_softmax=True,
                               
                               data_aug_meta_prefix='hlk2data', lbl2data_aug_meta_prefix='hlk2lbl', 
                               resize_length=5000,
                               
                               meta_loss_weight=0.3, pred_meta_prefix='cat', 
                               
                               fusion_loss_weight=0.05, use_fusion_loss=True, use_noise=True, use_encoder_parallel=False)
model.init_retrieval_head()
model.init_cross_head()

Some weights of RAD002 were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['encoder.cross_head.k.bias', 'encoder.cross_head.k.weight', 'encoder.cross_head.o.bias', 'encoder.cross_head.o.weight', 'encoder.cross_head.q.bias', 'encoder.cross_head.q.weight', 'encoder.cross_head.v.bias', 'encoder.cross_head.v.weight', 'encoder.dr_head.layer_norm.bias', 'encoder.dr_head.layer_norm.weight', 'encoder.dr_head.projector.bias', 'encoder.dr_head.projector.weight', 'encoder.dr_head.transform.bias', 'encoder.dr_head.transform.weight', 'encoder.meta_head.layer_norm.bias', 'encoder.meta_head.layer_norm.weight', 'encoder.meta_head.projector.bias', 'encoder.meta_head.projector.weight', 'encoder.meta_head.transform.bias', 'encoder.meta_head.transform.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model = model.to('cuda')

In [ ]:
b = prepare_batch(model, batch, m_args=[
    'pcat2data_idx', 'pcat2data_data2ptr', 'cat2data_idx', 'cat2data_input_ids', 'cat2data_attention_mask', 
    'cat2data_data2ptr',
    'pcat2lbl_idx', 'pcat2lbl_lbl2data2ptr', 'pcat2lbl_data2ptr', 'cat2lbl_idx', 'cat2lbl_input_ids', 
    'cat2lbl_attention_mask', 'cat2lbl_lbl2data2ptr', 'cat2lbl_data2ptr',
    
    'phlk2data_idx', 'phlk2data_data2ptr', 'hlk2data_idx', 'hlk2data_input_ids', 'hlk2data_attention_mask', 
    'hlk2data_data2ptr',
    'phlk2lbl_idx', 'phlk2lbl_lbl2data2ptr', 'phlk2lbl_data2ptr', 'hlk2lbl_idx', 'hlk2lbl_input_ids', 
    'hlk2lbl_attention_mask', 'hlk2lbl_lbl2data2ptr', 'hlk2lbl_data2ptr',
])

In [ ]:
b = prepare_batch(model, batch, m_args=[
    'phlk2data_idx', 'phlk2data_data2ptr', 'hlk2data_idx', 'hlk2data_input_ids', 'hlk2data_attention_mask', 
    'hlk2data_data2ptr',
    'phlk2lbl_idx', 'phlk2lbl_lbl2data2ptr', 'phlk2lbl_data2ptr', 'hlk2lbl_idx', 'hlk2lbl_input_ids', 
    'hlk2lbl_attention_mask', 'hlk2lbl_lbl2data2ptr', 'hlk2lbl_data2ptr',
])

In [ ]:
o = model(**b.to(model.device))

In [ ]:
o.loss

tensor(0.0776, grad_fn=<AddBackward0>)